In [3]:
import os
from glob import glob
import pickle

import joblib

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
data_dir = glob('../src/data/SeaFog/*.csv')

In [5]:
data_dir.sort()

In [6]:
data_dir

['../src/data/SeaFog/302_증산도.csv',
 '../src/data/SeaFog/771_불무기도.csv',
 '../src/data/SeaFog/772_장도.csv',
 '../src/data/SeaFog/789_목포구.csv',
 '../src/data/SeaFog/SF_0008_여수 해무관측소.csv']

In [7]:
col_list = ['지점 기온(°C)',
'수온(°C)',
'강수량 1분(mm)',
'풍속(m/s)',
'u',
'v',
"현지기압(hPa)",
'해면기압(hPa)',
'습도(%)',
'일사(MJ/m^2)',
'일조(Sec)',
'T',
'해기차(T-SST)',
'이슬점온도(°C)',
'기온',
'이슬점온도(T-Td)',
'수온',
'이슬점온도(°C)',
'vis']


In [8]:
col = ['기온(°C)',
 '수온(°C)',
 '풍속(m/s)',
 'u',
 'v',
 '습도(%)',
 '해기차(T-SST)',
 '이슬점온도(°C)',
 '기온-이슬점온도(T-Td)',
 '수온-이슬점온도(°C)',
 'vis']

In [9]:
new_col = ['t', 'st', 'ws', 'u', 'v', 'h', 'astd', 
          'dp', 't-dp', 'st-dp', 'vis']

In [10]:
temp = []

for ii, _dir in enumerate(data_dir):
    data = pd.read_csv(_dir, index_col=0, encoding='cp949')
    island_name = data['지점'][0]
    data = data[data.index >= '2019-12-14 09:00:00']
    data.drop('지점', axis=1, inplace=True)
    data.drop('해면기압(hPa)', axis=1, inplace=True)
    
    if ii < len(data_dir)-1:
        data.drop('일사(MJ/m^2)', axis=1, inplace=True)
        data.drop('일조(Sec)', axis=1, inplace=True)
        data.drop('1분 강수량(mm)', axis=1, inplace=True)
        data.drop('현지기압(hPa)', axis=1, inplace=True)
        data.drop('T', axis=1, inplace=True)
        
        df_1 = data[data.index <= '2021-06-01 23:50:00']
        df_2 = data[data.index >= '2021-07-01 00:00:00']
        
    
        data = pd.concat([df_1, df_2])
        
#     data.interpolate(inplace = True)

    col_name_change = {col[i]:str(island_name)+'_'+new_col[i] for i in range(len(new_col))}
    data.rename(columns=col_name_change, inplace=True)
        
#     if ii == len(data_dir)-1:
#         data.drop('해면기압(hPa)', axis=1, inplace=True)
    temp.append(data)

In [11]:
dataframe = pd.DataFrame()

for df in temp:
    dataframe = pd.concat([dataframe, df], axis=1)

In [12]:
dataframe

,302_t,302_st,302_ws,302_u,302_v,302_h,302_astd,302_dp,302_t-dp,302_st-dp,...,SF_0008_st,SF_0008_ws,SF_0008_u,SF_0008_v,SF_0008_h,SF_0008_astd,SF_0008_dp,SF_0008_t-dp,SF_0008_st-dp,SF_0008_vis
time,,,,,,,,,,,,,,,,,,,,,
2019-12-14 09:00:00,7.58,10.70,4.080000,0.382168,4.053507,66.18,-3.12,0.957286,6.622714,9.742714,...,11.900,1.966667,-1.580909,0.110309,67.888889,-3.033333,2.639491,6.227176,9.260509,3000.0
2019-12-14 09:10:00,7.67,10.70,4.240000,0.531895,4.198112,63.21,-3.03,0.316809,7.353191,10.383191,...,11.900,1.912500,-1.784218,0.154687,65.000000,-2.950000,2.024695,6.925305,9.875305,3000.0
2019-12-14 09:20:00,7.72,10.62,4.300000,0.599620,4.235989,61.02,-2.90,-0.187520,7.907520,10.807520,...,11.900,1.690000,-1.590688,0.263196,62.100000,-2.760000,1.489666,7.650334,10.410334,3000.0
2019-12-14 09:30:00,7.80,10.57,3.980000,1.161628,3.644537,60.19,-2.77,-0.320629,8.120629,10.890629,...,11.900,1.300000,0.007476,0.004024,61.750000,-2.650000,1.509457,7.740543,10.390543,3000.0
2019-12-14 09:40:00,7.79,10.50,4.388889,0.645240,4.337860,61.24,-2.71,-0.061281,7.851281,10.561281,...,11.900,3.510000,1.809175,2.474179,60.200000,-2.710000,1.053149,8.136851,10.846851,3000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-30 23:10:00,24.25,23.00,1.150000,0.502581,-0.958005,92.37,1.25,22.986038,1.263962,0.013962,...,22.850,0.890000,0.001973,-0.868320,100.000000,1.270000,24.124400,-0.004400,-1.274400,3000.0
2022-06-30 23:20:00,24.12,23.00,0.440000,0.127986,-0.258915,93.02,1.12,22.970495,1.149505,0.029505,...,22.851,0.180000,-0.085532,-0.156948,100.000000,1.339000,24.194400,-0.004400,-1.343400,3000.0
2022-06-30 23:30:00,24.19,23.00,0.980000,0.546869,-0.778959,92.75,1.19,22.993040,1.196960,0.006960,...,22.864,0.150000,-0.007961,-0.127845,100.000000,1.358222,24.226622,-0.004400,-1.362622,3000.0


In [13]:
dataframe.dropna(inplace=True)

In [14]:
dataframe

,302_t,302_st,302_ws,302_u,302_v,302_h,302_astd,302_dp,302_t-dp,302_st-dp,...,SF_0008_st,SF_0008_ws,SF_0008_u,SF_0008_v,SF_0008_h,SF_0008_astd,SF_0008_dp,SF_0008_t-dp,SF_0008_st-dp,SF_0008_vis
time,,,,,,,,,,,,,,,,,,,,,
2019-12-14 09:10:00,7.67,10.70,4.240000,0.531895,4.198112,63.21,-3.03,0.316809,7.353191,10.383191,...,11.900,1.9125,-1.784218,0.154687,65.00,-2.950000,2.024695,6.925305,9.875305,3000.0
2019-12-14 09:20:00,7.72,10.62,4.300000,0.599620,4.235989,61.02,-2.90,-0.187520,7.907520,10.807520,...,11.900,1.6900,-1.590688,0.263196,62.10,-2.760000,1.489666,7.650334,10.410334,3000.0
2019-12-14 09:30:00,7.80,10.57,3.980000,1.161628,3.644537,60.19,-2.77,-0.320629,8.120629,10.890629,...,11.900,1.3000,0.007476,0.004024,61.75,-2.650000,1.509457,7.740543,10.390543,3000.0
2019-12-14 09:40:00,7.79,10.50,4.388889,0.645240,4.337860,61.24,-2.71,-0.061281,7.851281,10.561281,...,11.900,3.5100,1.809175,2.474179,60.20,-2.710000,1.053149,8.136851,10.846851,3000.0
2019-12-14 09:50:00,7.90,10.48,4.220000,0.792587,4.134146,61.47,-2.58,0.105694,7.794306,10.374306,...,11.900,4.4000,1.216700,4.005781,58.70,-2.900000,0.475382,8.524618,11.424618,3000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-30 23:10:00,24.25,23.00,1.150000,0.502581,-0.958005,92.37,1.25,22.986038,1.263962,0.013962,...,22.850,0.8900,0.001973,-0.868320,100.00,1.270000,24.124400,-0.004400,-1.274400,3000.0
2022-06-30 23:20:00,24.12,23.00,0.440000,0.127986,-0.258915,93.02,1.12,22.970495,1.149505,0.029505,...,22.851,0.1800,-0.085532,-0.156948,100.00,1.339000,24.194400,-0.004400,-1.343400,3000.0
2022-06-30 23:30:00,24.19,23.00,0.980000,0.546869,-0.778959,92.75,1.19,22.993040,1.196960,0.006960,...,22.864,0.1500,-0.007961,-0.127845,100.00,1.358222,24.226622,-0.004400,-1.362622,3000.0
